In [1]:
!pip install llama_index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface-api
!pip install chromadb
!pip install llama-index-vector-stores-chroma

# Table of Contents
1. [Introduction](#introduction)
2. [Load Environment Variables](#load-environment-variables)
3. [Set up LLM and Embedding Model](#set-up-llm-and-embedding-model)
4. [Create In-memory Vector Store](#create-in-memory-vector-store)
5. [Create and Load On-disk Vector Store](#create-and-load-on-disk-vector-store)
6. [Update and Delete Data](#update-and-delete-data)


## Introduction

In this basic RAG (Retrieval-Augmented Generation) example, we take a Paul Graham essay, split it into chunks, embed it using an open-source embedding model, load it into Chroma, and then query it.

## Load Environment Variables from secrets

In this section, we will load the necessary environment variables required for accessing the Hugging Face LLMs and Embedding models. <br>

Get the Hugging Face Token: <br>
How to get your Hugging Face token: https://huggingface.co/docs/hub/en/security-tokens

In [ ]:
# Update me
# Access the Hugging Face environment variables
HF_TOKEN = ""

## Set up LLM and Embedding Model

In this section, we will set up the Hugging Face LLM and embedding model using the loaded environment variables.

In [3]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm_model = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=HF_TOKEN)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
Settings.llm_model.complete("To infinity, and")

CompletionResponse(text=" beyond!\n\nThe Toy Story franchise has been a beloved part of pop culture for over two decades, and it's not slowing down anytime soon. The latest installment, Toy Story 4, is set to hit theaters this summer, and it's already generating buzz.\n\nThe movie follows the adventures of Woody, Buzz, and the gang as they embark on a new adventure with a new toy, Forky. The trailer for the movie has been released, and it's already getting fans excited for the film.\n\nOne of the most exciting things about Toy Story 4 is the return of some beloved characters. Bo Peep, who was last seen in Toy Story 2, is back and looking better than ever. She's now a modern, independent woman, and her new look has been getting a lot of attention.\n\nAnother exciting addition to the movie is the introduction of new characters, including Forky, who is voiced by Tony Hale. Forky is a spork with a popsicle stick for a handle, and he's not exactly thrilled about being a toy.\n\nThe trailer 

## Create Vector Store (Chroma)

In this section, we will download the document, load the documents, create a Chroma vector store, and store the embedded documents.

In [5]:
# Import Chroma and other required libraries
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display

In [6]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-02-15 11:46:57--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.003s  

2025-02-15 11:46:57 (22.4 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [ ]:
# Update me
# Load documents
documents = SimpleDirectoryReader("/content/data/paul_graham/").load_data()

## Create and Load On-disk Vector Store

Extending the previous example, if you want to save to disk, simply initialize the Chroma client and pass the directory where you want the data to be saved to.

`Caution`: Chroma makes a best-effort to automatically save data to disk, however multiple in-memory clients can stomp each other's work. As a best practice, only have one client per path running at any given time.

In [10]:
# Save to disk

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=Settings.embed_model
)

# Load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=Settings.embed_model,
)

# Query Data from the persisted index
query_engine = index.as_query_engine(llm=Settings.llm_model)
response = query_engine.query("What did the author do growing up?")
display(Markdown(f"<b>{response}</b>"))

<b>

The author grew up writing short stories and programming on an IBM 1401 in 9th grade. In college, the author initially planned to study philosophy but switched to AI after being drawn into the world of Heinlein's novel, The Moon is a Harsh Mistress, and a PBS documentary featuring Terry Winograd using SHRDLU. The author learned Lisp to teach themselves AI and eventually wrote a book about Lisp hacking.</b>

## Update and Delete Data

While building toward a real application, you want to go beyond adding data, and also update and delete data.

Chroma has users provide `ids` to simplify the bookkeeping here. `ids` can be the name of the file, or a combined hash like `filename_paragraphNumber`, etc.

Here is a basic example showing how to do various operations:

In [11]:
doc_to_update = chroma_collection.get(limit=1)
doc_to_update["metadatas"][0] = {
    **doc_to_update["metadatas"][0],
    **{"author": "Paul Graham"},
}
chroma_collection.update(
    ids=[doc_to_update["ids"][0]], metadatas=[doc_to_update["metadatas"][0]]
)
updated_doc = chroma_collection.get(limit=1)
print(updated_doc["metadatas"][0])

# delete the last document
print("count before", chroma_collection.count())
chroma_collection.delete(ids=[doc_to_update["ids"][0]])
print("count after", chroma_collection.count())

{'_node_content': '{"id_": "4484ceec-fa2d-461b-8964-6765c521ae49", "embedding": null, "metadata": {"file_path": "/content/data/paul_graham/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2025-02-15", "last_modified_date": "2025-02-15"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "b58855ac-1a62-43d0-bd3f-b1589472356a", "node_type": "4", "metadata": {"file_path": "/content/data/paul_graham/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2025-02-15", "last_modified_date": "2025-02-15"}, "hash": "0c3c3f46cac874b495d944dfc4b920f6b68817dbbb1699ecc955d1fafb2bf87b", "class_name": "Rela